In [6]:
import pickle
from tqdm import tqdm
import numpy as np

# merge two or more existing pickle data sets|

In [59]:
seq_len = None # 2000

# data_path = '/Users/jplasser/Documents/AI Master/WS2021/MastersThesis/code.nosync/CNEP/src/data/mimic3/'
data_path = './'
data_path = '../data/mimic3/'

merge_datasets = ['train','val'] # , 'test']
target_dataset = 'extended'

dataset = merge_datasets[0]
template = f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle'
data = pickle.load(open(template, 'rb'))

for dataset in merge_datasets[1:]:
    template = f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle'
    data_ = pickle.load(open(template, 'rb'))

    for k in data.keys():
        if isinstance(data[k], np.ndarray):
            data[k] = np.concatenate((data[k], data_[k]), axis=0)
        else:
            data[k].extend(data_[k])
            
assert len(set([d.shape[0] if isinstance(d, np.ndarray) else len(d) for d in data.values()])) == 1

dataset = target_dataset
template = f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle'
pickle.dump(data, open(template, 'wb'))

# split into train and test set

In [45]:
from sklearn.model_selection  import train_test_split
import pandas as pd

In [53]:
test_size = 0.01
seed = 4711

X = data['inputs']
E = data['embeds']
Ee = data['embeds_events']
y = data['labels']

In [54]:
data.keys()

dict_keys(['labels', 'inputs', 'embeds', 'embeds_events'])

In [55]:
X_train, X_test, E_train, E_test, Ee_train, Ee_test, y_train, y_test = train_test_split(X, E, Ee, y, test_size=test_size, random_state=seed)

In [56]:
X_train.shape, E_train.shape, Ee_train.shape, y_train.shape, X_test.shape, E_test.shape, Ee_test.shape, y_test.shape

((21384, 48, 390),
 (21384, 700),
 (21384, 700),
 (21384,),
 (217, 48, 390),
 (217, 700),
 (217, 700),
 (217,))

In [57]:
data_train = {'labels':y_train, 'inputs': X_train, 'embeds': E_train, 'embeds_events': Ee_train}
data_test = {'labels':y_test, 'inputs': X_test, 'embeds': E_test, 'embeds_events': Ee_test}

In [58]:
dataset = f'train_split_{1-test_size}'
template = f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle'
pickle.dump(data_train, open(template, 'wb'))

dataset = f'test_split_{test_size}'
template = f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle'
pickle.dump(data_test, open(template, 'wb'))